<a href="https://colab.research.google.com/github/springboardmentor0327/Text_Summarization_Infosys_Internship_Oct2024/blob/Sameer/FINAL_gradio_app_with_all_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet
!pip install sumy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.2 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=274f2b045494d8398ecc2ce9aaca42ec4f7c6b2b7dff5e8996e89005998264d0
  Stored in directory: /root/.cache/pip/whe

**FREQUENCY MODEL**

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
# Download required resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  # For lemmatization
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
def freq_model(text, max_words=30):
    # Load English stopwords
    stopwords1 = set(stopwords.words("english"))

    # Tokenize text into words
    words = word_tokenize(text)

    # Create a frequency table for words (ignoring stopwords)
    freqTable = {word.lower(): words.count(word.lower()) for word in words if word.lower() not in stopwords1}

    # Tokenize text into sentences
    sentences = sent_tokenize(text)

    # Create a sentence value dictionary and sentence score
    sentenceValue = {}
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                sentenceValue[sentence] = sentenceValue.get(sentence, 0) + freq

    # Calculate the sum of sentence values
    sumValues = sum(sentenceValue.values())

    # Calculate the average sentence value
    average = int(sumValues / len(sentenceValue))

    # Generate the summary with max and min word constraints
    summary_sentences = []
    current_word_count = 0
    min_words = max_words - 10  # Default minimum words limit

    for sentence in sentences:
        if sentenceValue.get(sentence, 0) > (1.2 * average):
            sentence_word_count = len(word_tokenize(sentence))
            if min_words <= current_word_count + sentence_word_count <= max_words:
                summary_sentences.append(sentence)
                current_word_count += sentence_word_count
    summary = ' '.join(summary_sentences)
    summary_word_count = len(word_tokenize(summary))
    return summary,(f"\nWord Count (Summary): {summary_word_count}")

**SUMY MODEL**

In [5]:
!pip install sumy

In [6]:
# Import required libraries
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [7]:
def sumy_model(text, max_words=50):
    # Create a plaintext parser with English tokenization
    parser = PlaintextParser.from_string(text, Tokenizer("english"))

    # Create a TextRank summarizer
    summarizer = TextRankSummarizer()

    # Summarize the text, initially setting to retrieve more sentences than needed
    summary_sentences = summarizer(parser.document, len(parser.document.sentences))

    # Initialize variables for word count
    summary = []
    current_word_count = 0
    min_words = max_words - 10  # Default minimum word count

    # Iterate over sentences, adding them until we reach the max word limit
    for sentence in summary_sentences:
        sentence_word_count = len(word_tokenize(str(sentence)))
        if current_word_count + sentence_word_count > max_words:
            break  # Stop if adding the sentence exceeds max_words
        if current_word_count + sentence_word_count >= min_words:
            summary.append(str(sentence))
            current_word_count += sentence_word_count

    # Convert the summary list to a string
    text_summary = " ".join(summary)
    summary_word_count = len(word_tokenize(text_summary))
    return text_summary,(f"\nWord Count (Summary): {summary_word_count}")

**LUHN MODEL**

In [8]:
# Import required libraries
from sumy.summarizers.text_rank import TextRankSummarizer

In [9]:
def luhn_model(text, max_words=30):
    # Create a plaintext parser with English tokenization
    parser = PlaintextParser.from_string(text, Tokenizer("english"))

    # Create a Luhn summarizer
    summarizer_luhn = LuhnSummarizer()

    # Generate all sentences in the summary initially
    summary_sentences = summarizer_luhn(parser.document, len(parser.document.sentences))

    # Initialize variables for word count
    summary = []
    current_word_count = 0
    min_words = max(0, max_words - 10)  # Default minimum word count, ensures it’s non-negative

    # Iterate over sentences, adding them until we reach the max word limit
    for sentence in summary_sentences:
        sentence_word_count = len(word_tokenize(str(sentence)))

        # Check if adding this sentence would exceed max_words
        if current_word_count + sentence_word_count > max_words:
            break  # Stop if adding the sentence would exceed max_words

        # Add the sentence to summary and update the word count
        summary.append(str(sentence))
        current_word_count += sentence_word_count

        # Break if we’ve reached the minimum word count required and are close to max_words
        if current_word_count >= min_words:
            break

    # Convert the summary list to a single string
    final_summary = ' '.join(summary)
    summary_word_count = len(word_tokenize(final_summary))
    return final_summary,(f"\nWord Count (Summary): {summary_word_count}")

**T5 MODEL**

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

import glob
import pprint

pp = pprint.PrettyPrinter()

In [11]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
def t5_model(text,max_words):
    num_beams=5
    # Define min and max words for generation based on the provided max_words
    min_words = max(1, max_words - 10)
    max_tokens = max_words * 1.5  # Set max tokens with some buffer for BERT-based tokenization

    # Initialize the generated summary
    summary = ""

    # Repeat generation to ensure summary falls within the word count range
    while True:
        # Preprocess the text for encoding
        inputs = tokenizer.encode(
            "summarize: " + text,
            return_tensors='pt',
            max_length=512,  # Limit input length to avoid truncation
            truncation=True
        )

        # Generate the summary with length constraints
        summary_ids = model.generate(
            inputs,
            max_length=int(max_tokens),     # Set max tokens for output
            min_length=min_words,           # Set min length
            num_beams=num_beams,
            length_penalty=2.0,
            early_stopping=True
        )

        # Decode and calculate the word count of the generated summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary_words = summary.split()

        # Check if the summary is within the desired word range
        word_count = len(summary_words)
        if min_words <= word_count <= max_words:
            break

        # Adjust max_tokens slightly to retry if the summary doesn’t fit within the range
        max_tokens -= 5
        if max_tokens < min_words * 1.5:
            break  # Stop if max_tokens becomes impractically low
    summary_word_count = len(summary_words)
    return summary,(f"\nWord Count (Summary): {summary_word_count}")

**LLM MODEL**

In [13]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [14]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain langchain-google-genai langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 15.5 MB/s eta 0:00:00


In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

def load_llm(model="gemini-1.5-pro"):

  if model == "gemini-1.5-pro":
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2)
    return llm
  elif model == "gemini-1.5-flash":
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2)
    return llm
  else:
    raise ValueError("Invalid model name")

In [16]:
from langchain_core.prompts import ChatPromptTemplate

def get_prompt_template():
    # Define prompt with a strict word range instruction
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Write a concise summary of the following in {min_word_count} to {max_word_count} words. Stop once you reach the word limit:\n\n",
            ),
            ("human", "{context}")
        ]
    )
    return prompt

def llm_model(text, max_word_count=50):
    model="gemini-1.5-pro"
    # Set default for min_word_count if not provided
    # if min_word_count is None:
    min_word_count = max_word_count - 5

    # Load LLM
    llm = load_llm(model)

    # Get Prompt Template
    prompt = get_prompt_template()

    # Instantiate chain
    chain = prompt | llm

    # Invoke chain with specified range
    result = chain.invoke({
        "context": text,
        "min_word_count": min_word_count,
        "max_word_count": max_word_count
    })

    # Process output to enforce word limit
    output = result.content
    words = output.split()

    # If output exceeds max_word_count, truncate to max_word_count
    if len(words) > max_word_count:
        output = ' '.join(words[:max_word_count]) + "."

    # Return truncated output if necessary
    return output,(f"\nWord Count (Summary): {len(output.split())}")


**BART MODEL**

In [17]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [18]:
from transformers import BartTokenizer, BartForConditionalGeneration

def bart_model(article, max_words=50):
    # Load BART model and tokenizer
    model_name = 'facebook/bart-large-cnn'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Define minimum and maximum words for the summary
    min_words = max(10, max_words - 10)
    max_tokens = max_words * 1.5  # Adding a buffer as BERT-based tokens don't align 1:1 with words

    # Count the number of words in the original article
    original_word_count = len(article.split())
    print(f"Original article word count: {original_word_count}")

    # Tokenize and encode the article
    inputs = tokenizer.encode(article, return_tensors='pt', max_length=1024, truncation=True)

    # Generate the summary with specified length constraints
    summary = ""
    while True:
        summary_ids = model.generate(
            inputs,
            num_beams=4,
            min_length=min_words,           # Set minimum tokens
            max_length=int(max_tokens),      # Set maximum tokens
            length_penalty=2.0,
            early_stopping=True
        )

        # Decode and calculate word count of the summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary_word_count = len(summary.split())

        # Check if the summary is within the specified range
        if min_words <= summary_word_count <= max_words:
            break

        # Reduce max_tokens slightly and retry if summary is too long
        max_tokens -= 5
        if max_tokens < min_words * 1.5:  # Stop if max_tokens is too low to avoid an infinite loop
            break
    return summary,(f"Generated summary word count: {summary_word_count}")

In [20]:
!pip install PyPDF2
from PyPDF2 import PdfReader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.1 MB/s eta 0:00:00


**Text Extractor**

In [21]:
def extract_text_from_pdf(pdf_file):
    """Extract text from uploaded PDF file."""
    reader = PdfReader(pdf_file.name)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

**Refine and map_Reduce models**

In [37]:
import os
import asyncio
from typing import List
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from PyPDF2 import PdfReader
import gradio as gr

# Configure Google Gemini 1.5 Flash LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# # Utility to extract text from a PDF
# def extract_text_from_pdf(pdf_path: str) -> List[str]:
#     reader = PdfReader(pdf_path)
#     text = ""
#     for page in reader.pages:
#         text += page.extract_text()
#     return text.split("\n\n")  # Split into chunks (paragraphs)

# Iterative Refinement Components
summarize_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following: {context}")])
initial_summary_chain = summarize_prompt | llm | StrOutputParser()

refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_summary_chain = refine_prompt | llm | StrOutputParser()

# Iterative Refinement Function
async def iterative_refinement(chunks: List[str]) -> str:
    summary = await initial_summary_chain.ainvoke(chunks[0], {})
    for chunk in chunks[1:]:
        summary = await refine_summary_chain.ainvoke(
            {"existing_answer": summary, "context": chunk}, {}
        )
    return summary

# Map-Reduce Components
map_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following:\n\n{context}")])
map_chain = map_prompt | llm | StrOutputParser()

reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill them into a final, consolidated summary of the main themes.
"""
reduce_prompt = ChatPromptTemplate([("human", reduce_template)])
reduce_chain = reduce_prompt | llm | StrOutputParser()

# Map-Reduce Function
async def map_reduce_summarization(chunks: List[str]) -> str:
    summaries = []

    # Step 1: Generate summaries for each chunk (Map Step)
    for chunk in chunks:
        summary = await map_chain.ainvoke(chunk, {})
        summaries.append(summary)

    # Step 2: Consolidate the summaries into a final summary (Reduce Step)
    consolidated_summaries = "\n".join(summaries)
    final_summary = await reduce_chain.ainvoke({"docs": consolidated_summaries}, {})
    return final_summary


**Defining functions of all models**

In [38]:
# Placeholder for input and method-specific functions
def extractive_summarize(text, method, max_words):
    if method == "Frequency Method":
        return freq_model(text, max_words)
    elif method == "Sumy Method":
        return sumy_model(text, max_words)
    elif method == "Luhn Method":
        return luhn_model(text, max_words)

def abstractive_summarize(text, model, max_words):
    if model == "T5":
        return t5_model(text,max_words)
    elif model == "BART":
        return bart_model(text, max_words)


def advance_summarize(textt, model, max_words):
    if model == "LLM Model":
        return llm_model(textt, max_words)
    elif model == "refine":
      return iterative_refinement(textt),max_words
    elif method == "map_reduce":
      return map_reduce_summarization(textt),max_words
    # elif model == "refine":
    #     return llm_model(text, max_words)
    # elif model == "map_Reduce":
    #     return llm_model(text, max_words)

**Document Summarizer**

In [22]:
!pip install -U gradio langchain langchain-community pypdf langchain-google-genai langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00


In [23]:
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
      model="gemini-1.5-flash",
      temperature=0,
      max_tokens=None,
      timeout=None,
      max_retries=2
)


In [25]:
def summarize_pdf(pdf_file_path, chain_type, max_word_count):
    loader = PyPDFLoader(pdf_file_path)
    docs = loader.load_and_split()

    # Set minimum word count
    min_word_count = max_word_count - 10

    if chain_type == "map_Reduce":
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.invoke(docs)
    elif chain_type == "refine":
        chain = load_summarize_chain(llm, chain_type="refine")
        summary = chain.invoke(docs)
    else:
        return "Enter valid chain_type!! (map_reduce or refine)"

    # Ensure the summary is within the specified word count range
    summary_text = summary['output_text']
    words = summary_text.split()
    word_count = len(words)

    if word_count < min_word_count:
        # Expand the summary until it reaches the minimum word count
        while word_count < min_word_count:
            summary_text += " " + words[word_count % len(words)]
            words = summary_text.split()
            word_count = len(words)
    elif word_count > max_word_count:
        # Truncate the summary until it reaches the maximum word count
        summary_text = " ".join(words[:max_word_count])+'.'
        word_count = max_word_count  # Update word count to max_word_count

    return summary_text, (f"\nWord Count (Summary): {len(summary_text.split())}")  # Return both summary and word count

    # return summary_text

**Handling Functions**

In [26]:
def extractive_handler(pdf_file, input_text, method, word_count):
    text = input_text
    if pdf_file is not None:
        text = extract_text_from_pdf(pdf_file)
    return extractive_summarize(text, method, word_count)

In [27]:
def abstractive_handler(pdf_file, input_text, method, word_count):
    text = input_text
    if pdf_file is not None:
        text = extract_text_from_pdf(pdf_file)
    return abstractive_summarize(text, method, word_count)

In [28]:
def advance_handler(pdf_file, input_text, method, max_words):
    text = input_text
    if pdf_file is not None:
        if method == "map_Reduce":
            return summarize_pdf(pdf_file,method, max_words)
        elif method == "refine":
            return summarize_pdf(pdf_file,method, max_words)
        else:
            text = extract_text_from_pdf(pdf_file)
    return advance_summarize(text, method, max_words)

**Gradio App**

In [40]:
import gradio as gr

# Define CSS
css = """
h1 {
    margin-top: 2rem;
    font-size: 2rem;
    text-align: center;
}
"""
input_text = gr.Textbox(label="Input Text", lines=10)
max_words = gr.Slider(label="Max Words", minimum=10, maximum=100, step=10, value=50)
input_pdf_path = gr.File(label="Enter the PDF file path")
# chain_type = gr.Text(label="Enter Chain_type")
output_summary = gr.Text(label="Summary")


# Function to toggle input fields
def toggle_input(input_type):
    if input_type == "Text":
        return gr.update(visible=True), gr.update(visible=False)
    elif input_type == "PDF":
        return gr.update(visible=False), gr.update(visible=True)

# Gradio app structure
with gr.Blocks(title="Summarizer App", css=css) as demo:
    gr.Markdown("# Summarizer App")

    with gr.Tabs():
        with gr.TabItem("Extractive"):
            with gr.Row():
                # Left column: Inputs
                with gr.Column(scale=1):
                    gr.Markdown("### Inputs")
                    input_type = gr.Radio(
                        ["Text", "PDF"],
                        label="Select Input Type",
                        value="Text",
                    )

                    input_text = gr.Textbox(
                        label="Input Text",
                        lines=10,
                        visible=True,
                    )
                    input_pdf_path = gr.File(
                        label="Upload PDF",
                        file_types=[".pdf"],
                        visible=False,
                    )

                    extractive_dropdown = gr.Dropdown(
                        ["Frequency Method", "Sumy Method", "Luhn Method"],
                        label="Choose Extractive Method",
                    )

                    max_words = gr.Slider(
                        label="Max Words", minimum=10, maximum=100, step=10, value=40
                    )

                    summarize_button = gr.Button("Summarize")

                # Right column: Outputs
                with gr.Column(scale=1):
                    gr.Markdown("### Outputs")
                    output_summary = gr.Textbox(label="Summary Output")
                    output_word_count = gr.Textbox(label="Word Count")

            # Change visibility based on input type selection
            input_type.change(
                toggle_input,
                inputs=input_type,
                outputs=[input_text, input_pdf_path],
            )

            # Trigger summarization
            summarize_button.click(
                extractive_handler,
                inputs=[input_pdf_path,input_text, extractive_dropdown, max_words],
                outputs=[output_summary, output_word_count],
            )
        with gr.TabItem("Abstractive"):
            with gr.Row():
                # Left column: Inputs
                with gr.Column(scale=1):
                    gr.Markdown("### Inputs")
                    input_type = gr.Radio(
                        ["Text", "PDF"],
                        label="Select Input Type",
                        value="Text",
                    )

                    input_text = gr.Textbox(
                        label="Input Text",
                        lines=10,
                        visible=True,
                    )
                    input_pdf_path = gr.File(
                        label="Upload PDF",
                        file_types=[".pdf"],
                        visible=False,
                    )

                    extractive_dropdown = gr.Dropdown(
                        ["T5", "BART"],
                        label="Choose Extractive Method",
                    )

                    max_words = gr.Slider(
                        label="Max Words", minimum=10, maximum=100, step=10, value=40
                    )

                    summarize_button = gr.Button("Summarize")

                # Right column: Outputs
                with gr.Column(scale=1):
                    gr.Markdown("### Outputs")
                    output_summary = gr.Textbox(label="Summary Output")
                    output_word_count = gr.Textbox(label="Word Count")

            # Change visibility based on input type selection
            input_type.change(
                toggle_input,
                inputs=input_type,
                outputs=[input_text, input_pdf_path],
            )

            # Trigger summarization
            summarize_button.click(
                abstractive_handler,
                inputs=[input_pdf_path,input_text, extractive_dropdown, max_words],
                outputs=[output_summary, output_word_count],
            )
        with gr.TabItem("Advance LLM's"):
            with gr.Row():
                # Left column: Inputs
                with gr.Column(scale=1):
                    gr.Markdown("### Inputs")
                    input_type = gr.Radio(
                        ["Text", "PDF"],
                        label="Select Input Type",
                        value="Text",
                    )

                    input_text = gr.Textbox(
                        label="Input Text",
                        lines=10,
                        visible=True,
                    )
                    input_pdf_path = gr.File(
                        label="Upload PDF",
                        file_types=[".pdf"],
                        visible=False,
                    )

                    extractive_dropdown = gr.Dropdown(
                        ["LLM Model","refine","map_Reduce"],
                        label="Choose Extractive Method",
                    )

                    max_words = gr.Slider(
                        label="Max Words", minimum=10, maximum=100, step=10, value=40
                    )

                    summarize_button = gr.Button("Summarize")

                # Right column: Outputs
                with gr.Column(scale=1):
                    gr.Markdown("### Outputs")
                    output_summary = gr.Textbox(label="Summary Output")
                    output_word_count = gr.Textbox(label="Word Count")

            # Change visibility based on input type selection
            input_type.change(
                toggle_input,
                inputs=input_type,
                outputs=[input_text, input_pdf_path],
            )

            # Trigger summarization
            summarize_button.click(
                advance_handler,
                inputs=[input_pdf_path,input_text, extractive_dropdown, max_words],
                outputs=[output_summary, output_word_count],
            )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0efe2e2ed4028c573b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**GRADIO APP without pdf version**

In [30]:
# import gradio as gr

# # Define your CSS
# css = """
# h1 {
#     margin-top: 2rem;
#     font-size: 2rem;
#     text-align: center;
# }
# """

# # Define input fields
# input_text = gr.Textbox(label="Input Text", lines=10)
# max_words = gr.Slider(label="Max Words", minimum=10, maximum=100, step=10, value=50)
# input_pdf_path = gr.File(label="Enter the PDF file path")
# # chain_type = gr.Text(label="Enter Chain_type")
# output_summary = gr.Text(label="Summary")

# # Gradio app structure
# with gr.Blocks(title="Summarizer App", css=css) as demo:
#     gr.Markdown("# Summarizer App")

#     with gr.Tabs():
#         with gr.TabItem("Extractive"):
#           input_type = gr.Radio(
#                         ["Text Input", "PDF Upload"],
#                         label="Input Type",
#                         value="Text Input"
#                     )

#                     # Text Input
#                     text_input = gr.Textbox(
#                         label="Input Text",
#                         lines=5,
#                         placeholder="Enter text here",
#                         visible=True
#                     )

#                     # PDF Upload
#                     pdf_input = gr.File(
#                         label="Upload PDF",
#                         type="filepath",
#                         file_types=[".pdf"],
#                         visible=False
#                     )
#             extractive_dropdown = gr.Dropdown(
#                 ["Frequency Method", "Sumy Method", "Luhn Method"],
#                 label="Choose Extractive Method",
#             )
#             gr.Interface(
#                 fn=extractive_summarize,
#                 inputs=[input_text, extractive_dropdown, max_words],
#                 # outputs=["text","text"],
#                 outputs=[
#                     gr.Textbox(label="Summary Output"),
#                     gr.Textbox(label="Word Count"),
#                 ],
#                 flagging_mode="never",
#                 live=False,
#             )
#         with gr.TabItem("Abstractive"):
#             abstractive_dropdown = gr.Dropdown(
#                 ["T5", "BART", "LLM Model"],
#                 label="Choose Abstractive Model",
#             )
#             gr.Interface(
#                 fn=abstractive_summarize,
#                 inputs=[input_text, abstractive_dropdown, max_words],
#                 # outputs="text",
#                 outputs=[
#                     gr.Textbox(label="Summary Output"),
#                     gr.Textbox(label="Word Count"),
#                 ],
#                 flagging_mode="never",
#                 live=False,
#             )
#         with gr.TabItem("PDF"):
#               chain_type = gr.Dropdown(
#                 ["refine", "map_reduce"],
#                 label="Choose Chain_type",
#             )
#               gr.Interface(fn=summarize_pdf,
#                         inputs=[input_pdf_path,chain_type,max_words],
#                         # outputs=[output_summary],
#                         outputs=[
#                            gr.Textbox(label="Summary Output"),
#                            gr.Textbox(label="Word Count"),
#                         ],
#                         flagging_mode='never',
#                         submit_btn='Generate',
#                         live=False,
#             )

# # Launch app
# demo.launch(debug=True)
